## **FINAL PROJECT TSDW** **Kelompok 8**

# **A. PENDAHULUAN**


---

**1. Deskripsi Dataset**

Dataset yang digunakan adalah Hotel Bookings Dataset. Data ini mencakup ribuan catatan pemesanan dari dua tipe properti: Resort Hotel (H1) dan City Hotel (H2).

- Tujuan Data: Menganalisis faktor-faktor yang mempengaruhi permintaan pemesanan
dan status akhirnya.

- Variabel Utama: Dataset ini memiliki 32 variabel yang mencakup:

    - Informasi Status: Status akhir pemesanan (is_canceled), status reservasi terakhir (reservation_status), dan tanggal status terakhir (reservation_status_date).

    - Waktu Pemesanan: Waktu tunggu (lead_time), tahun/bulan/minggu/hari kedatangan (arrival_date_year, arrival_date_month, dsb.).

    - Detail Tamu: Jumlah orang (adults, children, babies), tipe tamu (customer_type), dan negara asal (country).

    - Finansial & Logistik: Rata-rata Harga Harian (adr), tipe deposit (deposit_type), dan jumlah ruang parkir yang dibutuhkan (required_car_parking_spaces).

    - Agen & Perusahaan: Terdapat ID unik untuk agen (agent) dan perusahaan (company) yang membuat pemesanan.


**2. Alasan Dataset Menarik**

Dataset ini sangat ideal untuk tugas Data Wrangling karena kompleksitas dan tantangan yang dapat diselesaikan:

- Penanganan Nilai Hilang Kritis: Kolom agent dan company seringkali memiliki nilai hilang yang harus diatasi dengan metode imputasi atau pengkategorian yang tepat.

- Analisis Waktu/Tanggal: Memerlukan konversi variabel tanggal (arrival_date_year, arrival_date_month, dsb.) menjadi format datetime tunggal untuk analisis tren musiman yang efisien.

- Variabel Kategorikal Detail: Terdapat variabel-variabel dengan kategori rinci (misalnya market_segment, distribution_channel, meal) yang memerlukan standarisasi dan pembersihan (misalnya, menggabungkan kategori "Undefined/SC" pada kolom meal).

- Analisis Perubahan: Terdapat variabel untuk melacak perubahan pemesanan (booking_changes) dan riwayat tamu (is_repeated_guest, previous_cancellations) yang dapat digunakan untuk feature engineering.

**3. Pertanyaan/Tujuan Analisis**

Tujuan dari Data Wrangling ini adalah mempersiapkan data untuk menjawab:

1. Bagaimana rata-rata Harga Harian (adr) bervariasi antara City Hotel dan Resort Hotel berdasarkan musim kedatangan?

2. Variabel manakah (misalnya lead_time, deposit_type, atau booking_changes) yang paling berkorelasi dengan status pembatalan (is_canceled)?

3. Bagaimana missing values di kolom company dan agent dapat diolah sedemikian rupa sehingga tetap memberikan insight dalam analisis akhir?

# **B. Setup & Package**

In [1]:
# Import Library
# Standard Libraries for Data Manipulation
import pandas as pd
import numpy as np

# Libraries for Visualization (akan digunakan di Stage 4)
import matplotlib.pyplot as plt
import seaborn as sns

# Untuk set opsi pandas agar semua kolom terlihat
pd.set_option('display.max_columns', None)

# Import os agar bisa bikin folder
import os

**Penjelasan singkat fungsi paket**
- Pandas (pd): Manipulasi dan analisis data utama menggunakan DataFrame.

- NumPy (np): Untuk operasi numerik yang efisien, terutama perhitungan statistik.

- Matplotlib (plt): Membuat visualisasi statis dasar (seperti plot dan histogram).

- Seaborn (sns): Membuat grafik statistik tingkat tinggi dan lebih informatif (berbasis Matplotlib).

In [2]:
# Mendefinisikan Path (relative path dari folder 'notebooks/')
RAW_DATA_PATH = '../data/raw/hotels.csv'

# Path untuk menyimpan data yang sudah bersih
PROCESSED_DATA_PATH = '../data/processed/cleaned_hotel_bookings.csv'

# Path data mentah saat di Colab (jika di-upload langsung)
# Ganti ini jika Anda menggunakan Google Drive atau Colab lokal
try:
    df = pd.read_csv(RAW_DATA_PATH)
except FileNotFoundError:
    print("Menggunakan path cadangan (jika data di root Colab). Mohon perbaiki struktur di GitHub.")
    df = pd.read_csv("hotels.csv")

# Tampilkan 5 baris pertama untuk inspeksi
print("5 Baris Pertama Data Mentah:")
display(df.head())

Menggunakan path cadangan (jika data di root Colab). Mohon perbaiki struktur di GitHub.
5 Baris Pertama Data Mentah:


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


## **C. Data Preparation**
Mengubah dataset mentah (raw) menjadi dataset yang **bersih, konsisten, dan siap untuk analisis & sampling**

### ---------- INSPEKSI AWAL & INFO DATA ----------
Melihat struktur data, tipe kolom, dan jumlah nilai yang hilang (NA).

In [3]:
# Inspeksi Awal (Shape, Info, Head)
print(f"Ukuran dataset (baris, kolom): {df.shape}\n")
print("Informasi Tipe Data dan Non-Null:")
df.info()

Ukuran dataset (baris, kolom): (119390, 32)

Informasi Tipe Data dan Non-Null:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11 

### ---------- REMOVE DUPLICATES ----------
Menghilangkan baris yang 100% identik. Data yang kita gunakan adalah log pemesanan, sehingga duplikasi dapat diartikan sebagai kesalahan pencatatan atau entri ganda.

In [4]:
# Cek jumlah baris duplikat
total_duplikat = df.duplicated().sum()
print(f"Jumlah baris duplikat: {total_duplikat}")

# Menghapus Data Duplikat
if total_duplikat > 0:
    # Definisikan df_clean dari data yang sudah di-deduplicate
    df_clean = df.drop_duplicates(keep='first').reset_index(drop=True)
    print(f"Berhasil menghapus {total_duplikat} baris duplikat")
    print(f"Ukuran data setelah dibersihkan: {df_clean.shape}")
else:
    df_clean = df.copy()
    print("Tidak ditemukan data duplikat")

# Mulai sekarang, semua operasi harus menggunakan df_clean

Jumlah baris duplikat: 31994
Berhasil menghapus 31994 baris duplikat
Ukuran data setelah dibersihkan: (87396, 32)


### ---------- RENAME COLUMNS ----------
Mengubah nama kolom menjadi **snake_case** (lowercase + underscore) untuk standar Python.

In [5]:
# Mengganti nama kolom pada df_clean
df_clean.columns = (
    df_clean.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
)
print("Columns setelah rename:")
print(df_clean.columns.tolist())

Columns setelah rename:
['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status', 'reservation_status_date']


### ---------- MISSING VALUE IMPUTATION ----------
Menangani nilai kosong agar tidak mengganggu analisis.
- Kolom **numerik/ID** → isi dengan **0** (untuk `agent` dan `company` yang artinya "Tidak ada ID agen/perusahaan yang terdaftar").
- Kolom **kategorikal** (`country`) → isi dengan **'OTHERS'** (karena persentase NA kecil).

Setelah imputasi, **total missing value = 0**.

In [6]:
# Cek missing values pada df_clean
missing_values = df_clean.isnull().sum()
missing_percentage = (missing_values / len(df_clean) * 100).round(2)

missing_df = pd.DataFrame({
    'Jumlah NA': missing_values,
    'Persentase NA (%)': missing_percentage
}).sort_values(by='Persentase NA (%)', ascending=False)

print("Analisis Missing Value (NA > 0):")
display(missing_df[missing_df['Jumlah NA'] > 0])

Analisis Missing Value (NA > 0):


,Jumlah NA,Persentase NA (%)
company,82137,93.98
agent,12193,13.95
country,452,0.52
children,4,0.00


In [7]:
# 1. Imputasi ID Numerik (agent, company) dengan 0
# 0 menunjukkan tidak ada ID agen/perusahaan yang dicatat.
# Kita isi dengan 0 terlebih dahulu karena tipe data awalnya float/numerik.
df_clean['agent'].fillna(0, inplace=True)
df_clean['company'].fillna(0, inplace=True)

# 2. Imputasi Kategorikal (country) dengan 'OTHERS'
df_clean['country'].fillna('OTHERS', inplace=True)

# 3. Imputasi baris minor dengan mode/median
# Kolom yang hanya 1-2 NA (misal: adr, children, reservation_status_date)
for col in df_clean.columns:
    if df_clean[col].isnull().any():
        if df_clean[col].dtype in ['float64', 'int64']:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
        elif df_clean[col].dtype == 'object':
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

print("\nMissing value handling selesai")
print("Total missing setelah imputasi:", df_clean.isnull().sum().sum())

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20560\3990759995.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['agent'].fillna(0, inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20560\3990759995.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,


Missing value handling selesai
Total missing setelah imputasi: 0


### ---------- DATA TYPE CONVERSION & CLEANING ----------
Memperbaiki format data, menangani anomali, dan mengubah tipe data.

Langkah-langkah:
1. **Cleaning kolom `adr`**: Nilai negatif di-clip menjadi 0.
2. **Konversi ID & Jumlah Tamu**: `children`, `agent`, `company` → `int` (harus bilangan bulat).
3. **Konversi Tanggal**: `reservation_status_date` → `datetime` (diperbaiki format *parsing*-nya).

In [8]:
# Clean kolom adr (Hanya clip nilai negatif, asumsi tipe data sudah float dari imputasi)
df_clean['adr'] = df_clean['adr'].clip(lower=0)

# Konversi ke integer (setelah imputasi 0)
df_clean['children'] = df_clean['children'].astype(int)
df_clean['agent'] = df_clean['agent'].astype(int)
df_clean['company'] = df_clean['company'].astype(int)

# Konversi tanggal reservasi (Perbaikan: membiarkan Pandas mendeteksi format)
df_clean['reservation_status_date'] = pd.to_datetime(
    df_clean['reservation_status_date'],
    errors='coerce' # Memungkinkan deteksi format YYYY-MM-DD
)

print("Data type conversion & cleaning selesai")
print("Sample dtypes:")
print(df_clean[['adr', 'children', 'agent', 'reservation_status_date']].dtypes)

Data type conversion & cleaning selesai
Sample dtypes:
adr                               float64
children                            int32
agent                               int32
reservation_status_date    datetime64[ns]
dtype: object


### ---------- FEATURE ENGINEERING ----------
Membuat kolom baru yang lebih informatif & berguna untuk analisis.

Fitur baru yang ditambahkan:
- `arrival_date` → kolom datetime lengkap (dari year, month, day)
- `total_stays` → total malam menginap (weekend + weekdays)

In [9]:
# Mapping bulan
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Kolom arrival_date (datetime)
# Pastikan nilai month sudah di-map dan diubah ke int sebelum dikonversi
df_clean['arrival_date_month'] = df_clean['arrival_date_month'].map(month_map)

df_clean['arrival_date'] = pd.to_datetime({
    'year': df_clean['arrival_date_year'],
    'month': df_clean['arrival_date_month'],
    'day': df_clean['arrival_date_day_of_month']
})

# Total malam menginap
df_clean['total_stays'] = df_clean['stays_in_weekend_nights'] + df_clean['stays_in_week_nights']

print("Feature engineering selesai:")
print("- arrival_date (datetime)")
print("- total_stays (int)")

Feature engineering selesai:
- arrival_date (datetime)
- total_stays (int)


### ---------- STANDARISASI KATEGORI ----------
Menghilangkan inkonsistensi penulisan.

Perubahan:
- `meal`: `'Undefined'` → `'SC'` (Self Catering / tanpa paket makan)
- `market_segment`: `'Undefined'` → `'Other'`

In [10]:
# Standarisasi meal
df_clean['meal'] = df_clean['meal'].replace('Undefined', 'SC')

# Standarisasi market_segment
df_clean['market_segment'] = df_clean['market_segment'].replace('Undefined', 'Other')

print("Standarisasi kategori selesai")
print("Unique meal:", df_clean['meal'].unique())
print("Unique market_segment:", df_clean['market_segment'].unique())

Standarisasi kategori selesai
Unique meal: ['BB' 'FB' 'HB' 'SC']
Unique market_segment: ['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary' 'Groups'
 'Other' 'Aviation']


### ---------- FINAL CHECK DAN SIMPAN ----------
Verifikasi akhir + menyimpan hasil untuk Stage selanjutnya.

Hasil akhir yang diharapkan:
- Missing value: **0**
- File disimpan di: `data/processed/cleaned_hotel_bookings.csv`

In [11]:
# Definisikan path yang benar untuk diakses dari folder 'notebooks/'
PROCESSED_DATA_DIR = '../data/processed'
OUTPUT_FILE_PATH = os.path.join(PROCESSED_DATA_DIR, 'cleaned_hotel_bookings.csv')

# 1. Pastikan folder 'data/processed' dibuat
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
# 2. Simpan DataFrame yang sudah bersih
df_clean.to_csv(OUTPUT_FILE_PATH, index=False)

# --- Final Validation Check ---
# Recalculate total missing values in df_clean
total_na = df_clean.isnull().sum().sum()
duplicate_rows = df_clean.duplicated().sum()

print(f"\nFINAL DATASET")
print(f"Shape            : {df_clean.shape}")
print(f"Duplikat         : {duplicate_rows}")

if total_na > 0:
    print(f"Missing values   : {total_na}")
    print("\n⚠️ PERINGATAN: Masih ada Missing Value!")
    print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
else:
    print(f"Missing values   : 0")
    print("✅ Missing values = 0.")

print(f"\nSELESAI! Dataset bersih disimpan di:\n{OUTPUT_FILE_PATH}")

PermissionError: [WinError 5] Access is denied: '../data'

# **D. Exploratory Data Analysis**

---

In [55]:
# Import modul Bokeh yang diperlukan
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.io import output_notebook # Agar plot tampil di Jupyter Notebook
from math import pi, cos, sin
from bokeh.transform import cumsum

# Tampilkan output di notebook
output_notebook()

Loading BokehJS ...

In [59]:
# =======================================================================
# VISUALISASI : Proporsi Pemesanan Dibatalkan vs. Tidak Dibatalkan
# =======================================================================

# 1. Persiapan Data
cancel_counts = df['is_canceled'].value_counts().reset_index()
cancel_counts.columns = ['is_canceled', 'count']
cancel_counts['status_label'] = cancel_counts['is_canceled'].map({0: 'Tidak Dibatalkan', 1: 'Dibatalkan'})

# Hitung persentase dan angle (sudut untuk chart)
total = cancel_counts['count'].sum()
cancel_counts['percentage'] = (cancel_counts['count'] / total) * 100
# Hitung angle (sudut) untuk setiap slice dalam radian (2*pi)
cancel_counts['angle'] = cancel_counts['count'] / total * 2 * pi 
# Buat kolom warna
cancel_counts['color'] = ['#5cb85c', '#d9534f'] # Hijau untuk Tidak Dibatalkan, Merah untuk Dibatalkan

# Tambahkan kolom label persentase (untuk legend)
cancel_counts['legend_label'] = cancel_counts.apply(
    lambda row: f"{row['status_label']} ({row['percentage']:.2f}%)", axis=1
)

source_donut = ColumnDataSource(cancel_counts)

# 2. Membuat Figure
p1 = figure(
    height=350, 
    width=600, 
    title="Proporsi Pemesanan Dibatalkan vs. Tidak Dibatalkan",
    tools="hover, save",
    tooltips="@legend_label",
    x_range=(-0.5, 1.0)
)

# 3. Menambahkan Glyph (Wedge - Donut Chart)
p1.wedge(x=0, y=1, radius=0.3, # Radius donut
         start_angle=cumsum('angle', include_zero=True),
         end_angle=cumsum('angle'),
         line_color="white", 
         fill_color='color', 
         legend_field='legend_label', 
         source=source_donut)

# 4. Styling
p1.axis.visible = False # Sembunyikan sumbu X dan Y
p1.grid.grid_line_color = None
p1.toolbar_location = None

show(p1)

### Deskripsi Pie Chart

Pie chart di atas menggambarkan **proporsi status pemesanan** antara pesanan yang berakhir **Dibatalkan** dan yang **Tidak Dibatalkan**. 

- **Tidak Dibatalkan: 62.96%**  
  Bagian ini merupakan porsi terbesar, menunjukkan bahwa sebagian besar pemesanan berhasil direalisasikan.

- **Dibatalkan: 37.04%**  
  Nilai ini menunjukkan bahwa lebih dari sepertiga total pemesanan berakhir dengan pembatalan, sehingga menjadi indikator penting untuk evaluasi performa sistem reservasi.

Secara keseluruhan, visualisasi ini memberikan gambaran cepat mengenai tingkat pembatalan dan keberhasilan pemesanan, serta dapat membantu dalam mengidentifikasi potensi masalah atau area yang memerlukan analisis lebih lanjut.


In [74]:
# =======================================================================
# VISUALISASI 2: TREN PEMESANAN BERDASARKAN BULAN (LINE PLOT)
# =======================================================================

# 1. Persiapan Data (
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

monthly_bookings = df.groupby('arrival_date_month')['hotel'].count().reindex(month_order).reset_index(name='Total Pemesanan')
monthly_bookings['label_text'] = monthly_bookings['Total Pemesanan'].astype(str)

source_monthly = ColumnDataSource(monthly_bookings)

# 2. Membuat Figure 
p2 = figure(
    x_range=month_order, 
    height=400, 
    width=800, 
    title="Total Pemesanan Berdasarkan Bulan",
    tools="pan, wheel_zoom, box_zoom, reset, save, hover",
    tooltips=[
        ("Bulan", "@arrival_date_month"),
        ("Pemesanan", "@Total Pemesanan")
    ]
)

# 3. Menambahkan Glyph (Line dan Titik)
LINE_COLOR = "#00AEEF" 

p2.line(x='arrival_date_month', y='Total Pemesanan', source=source_monthly, 
        line_width=3, 
        color=LINE_COLOR) 
        
p2.scatter(x='arrival_date_month', y='Total Pemesanan', source=source_monthly, 
          marker='circle', # Menentukan jenis marker
          size=10, 
          color=LINE_COLOR, 
          fill_color="white", 
          line_width=2)

# Menambahkan Label Jumlah Pemesanan di Atas Titik
p2.text(x='arrival_date_month', y='Total Pemesanan', 
        text='label_text', 
        text_align="center", 
        text_baseline="bottom",
        source=source_monthly,
        text_font_size="8pt",
        y_offset=10) 

# 4. Styling 
p2.xaxis.major_label_orientation = 0.8 
p2.xaxis.axis_label = "Kedatangan"
p2.yaxis.axis_label = "Total Pemesanan"
p2.y_range.start = 0
p2.grid.grid_line_dash = [6, 4]
p2.grid.grid_line_alpha = 0.4

show(p2)

### Deskripsi Grafik: Total Pemesanan Berdasarkan Bulan

Grafik garis di atas menunjukkan **perkembangan total pemesanan per bulan** sepanjang tahun. Secara umum, terlihat adanya pola musiman (seasonal pattern) dengan peningkatan dan penurunan jumlah pemesanan pada periode tertentu.

**Ringkasan pola utama:**
- **Kenaikan Awal Tahun:** Jumlah pemesanan meningkat dari Januari hingga mencapai puncak kecil pada Mei.
- **Puncak Musiman:** Pemesanan mencapai titik tertinggi pada **Agustus**, menandakan periode dengan permintaan tertinggi.
- **Penurunan Akhir Tahun:** Setelah Agustus, pemesanan mulai menurun, terutama pada **November dan Desember**, yang menjadi periode dengan pemesanan paling rendah setelah Januari.

**Interpretasi Umum:**
Grafik ini menunjukkan adanya pola permintaan yang fluktuatif sepanjang tahun. Puncak pada pertengahan tahun dapat menunjukkan periode liburan atau musim ramai (peak season), sementara penurunan menjelang akhir tahun mungkin terkait dengan faktor musiman, operasional, atau perubahan preferensi pelanggan.

Visualisasi ini membantu mengidentifikasi bulan-bulan dengan permintaan tinggi serta periode yang perlu strategi optimasi kapasitas atau promosi tambahan.


In [75]:
# =======================================================================
# VISUALISASI 3: DISTRIBUSI ADR PER TIPE HOTEL
# =======================================================================

# 1. Persiapan Data: Menghitung Statistik Box Plot
hotels = ['City Hotel', 'Resort Hotel']

# Hitung statistik kuartil per tipe hotel
stats_df = df.groupby('hotel')['adr'].agg(
    q1=lambda x: x.quantile(0.25), # Menggunakan lambda
    q2='median',                 
    q3=lambda x: x.quantile(0.75), # Menggunakan lambda
    iqr=lambda x: x.quantile(0.75) - x.quantile(0.25)
).reset_index()

# Hitung batas atas dan bawah whiskers (1.5 * IQR)
stats_df['upper'] = stats_df['q3'] + 1.5 * stats_df['iqr']
stats_df['lower'] = stats_df['q1'] - 1.5 * stats_df['iqr']

source_box = ColumnDataSource(stats_df)

# 2. Membuat Figure
p3 = figure(
    x_range=hotels, 
    height=400, 
    width=600, 
    title="Distribusi Harga Harian Rata-rata (ADR) per Tipe Hotel",
    tools="pan, wheel_zoom, box_zoom, reset, save, hover",
    tooltips=[
        ("Hotel", "@hotel"),
        ("Median ADR", "@q2{0.2f}"),
        ("IQR", "@iqr{0.2f}")
    ]
)

# 3. Menambahkan Glyph (Box Plot Manual)
HOTEL_COLOR = Category10[3] 

# Batang IQR (Interquartile range: Q1 ke Q3)
p3.vbar(x='hotel', top='q3', bottom='q2', width=0.7, source=source_box, 
        line_color='black', fill_color=factor_cmap('hotel', palette=[HOTEL_COLOR[0], HOTEL_COLOR[1]], factors=hotels))
p3.vbar(x='hotel', top='q2', bottom='q1', width=0.7, source=source_box, 
        line_color='black', fill_color=factor_cmap('hotel', palette=[HOTEL_COLOR[0], HOTEL_COLOR[1]], factors=hotels))

# Whiskers (Garis T dari Q3 ke Upper dan Q1 ke Lower)
p3.segment(x0='hotel', y0='upper', x1='hotel', y1='q3', line_color="black", source=source_box)
p3.segment(x0='hotel', y0='lower', x1='hotel', y1='q1', line_color="black", source=source_box)

# Tops and Bottoms Whiskers (Garis horizontal di ujung T)
p3.rect(x='hotel', y='upper', width=0.2, height=0.01, line_color="black", fill_color="black", source=source_box)
p3.rect(x='hotel', y='lower', width=0.2, height=0.01, line_color="black", fill_color="black", source=source_box)

# 4. Styling
p3.xgrid.grid_line_color = None
p3.yaxis.axis_label = "Average Daily Rate (ADR)"
p3.yaxis.formatter = NumeralTickFormatter(format="0,0.00") 
p3.y_range.start = 0

show(p3)

### Deskripsi Grafik: Distribusi Harga Harian Rata-rata (ADR) per Tipe Hotel

Boxplot di atas menunjukkan **perbandingan distribusi Average Daily Rate (ADR)** antara dua tipe hotel: **City Hotel** dan **Resort Hotel**. Visualisasi ini membantu memahami perbedaan pola harga, variasi, serta potensi outlier pada masing-masing kategori.

**Ringkasan Temuan Utama:**

- **City Hotel**
  - Median ADR berada pada kisaran menengah.
  - Sebaran data relatif lebih rapat dibandingkan Resort Hotel.
  - Rentang interkuartil (IQR) lebih kecil, menandakan variasi harga yang lebih stabil.
  - Terdapat beberapa nilai rendah mendekati 0 yang dapat menjadi indikasi diskon besar atau tipe kamar berharga sangat rendah.

- **Resort Hotel**
  - Median ADR sedikit lebih tinggi dibandingkan City Hotel.
  - Sebaran nilai ADR lebih lebar, menunjukkan variasi harga yang lebih besar.
  - Rentang interkuartil (IQR) lebih luas, mencerminkan ketidakseragaman harga antar pemesanan.
  - Terdapat nilai ADR sangat tinggi yang berfungsi sebagai outlier, kemungkinan berasal dari kategori kamar premium atau musim puncak.

**Interpretasi Umum:**
Distribusi ADR pada Resort Hotel cenderung lebih beragam dengan kemungkinan harga premium lebih tinggi, sedangkan City Hotel menunjukkan pola harga yang lebih stabil. Perbedaan ini dapat mencerminkan karakteristik pasar, jenis layanan, serta faktor musiman pada masing-masing tipe hotel.


## Tabel Ringkasan – ADR & Durasi Menginap
Tabel ini menunjukkan rata-rata ADR dan lama menginap berdasarkan tipe hotel dan status pembatalan.


In [76]:
# =======================================================================
# TABEL RINGKASAN: RATA-RATA ADR & DURASI MENGINAP
# =======================================================================

# Pastikan kolom 'total_nights' sudah dibuat
if 'total_nights' not in df.columns:
    df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

# Hitung metrik ringkasan
adr_nights_summary = df.groupby(['hotel', 'is_canceled']).agg(
    Avg_ADR=('adr', 'mean'),
    Median_Total_Nights=('total_nights', 'median'),
    Total_Bookings=('is_canceled', 'count')
).reset_index()

# Ubah nilai 'is_canceled' menjadi label yang lebih deskriptif
adr_nights_summary['Status'] = adr_nights_summary['is_canceled'].map({0: 'Tidak Dibatalkan', 1: 'Dibatalkan'})
adr_nights_summary['Rata-rata ADR'] = adr_nights_summary['Avg_ADR'].round(2)
adr_nights_summary = adr_nights_summary.drop(columns=['is_canceled', 'Avg_ADR'])


print("\n--- Tabel Ringkasan: Rata-Rata ADR & Durasi Menginap ---")
print(adr_nights_summary.rename(columns={
                                         'hotel': 'Tipe Hotel', 
                                         'Median_Total_Nights': 'Median Malam Menginap',
                                         'Total_Bookings': 'Total Pemesanan'}).to_markdown(index=False))


--- Tabel Ringkasan: Rata-Rata ADR & Durasi Menginap ---
| Tipe Hotel   |   Median Malam Menginap |   Total Pemesanan | Status           |   Rata-rata ADR |
|:-------------|------------------------:|------------------:|:-----------------|----------------:|
| City Hotel   |                       3 |             46228 | Tidak Dibatalkan |          105.75 |
| City Hotel   |                       3 |             33102 | Dibatalkan       |          104.69 |
| Resort Hotel |                       3 |             28938 | Tidak Dibatalkan |           90.79 |
| Resort Hotel |                       4 |             11122 | Dibatalkan       |          105.79 |


In [77]:
# =======================================================================
# VISUALISASI 4: DISTRIBUSI LEAD TIME BERDASARKAN STATUS PEMBATALAN (FIX)
# =======================================================================

# 1. Persiapan Data: Menghitung Statistik Box Plot untuk 'lead_time'
df_lt = df[df['lead_time'] < 400].copy() 

# Tambahkan kolom label
df_lt.loc[:, 'status_label'] = df_lt['is_canceled'].map({0: 'Tidak Dibatalkan', 1: 'Dibatalkan'})
statuses = ['Tidak Dibatalkan', 'Dibatalkan']

# Hitung statistik Box Plot
stats_lt = df_lt.groupby('status_label')['lead_time'].agg(
    q1=lambda x: x.quantile(0.25),
    q2='median',
    q3=lambda x: x.quantile(0.75),
    iqr=lambda x: x.quantile(0.75) - x.quantile(0.25)
).reset_index()

# Hitung batas atas dan bawah whiskers (1.5 * IQR)
stats_lt['upper'] = stats_lt['q3'] + 1.5 * stats_lt['iqr']
stats_lt['lower'] = stats_lt['q1'] - 1.5 * stats_lt['iqr']

# Batas agar whisker tidak melewati batas data maksimum/minimum
stats_lt['max_data'] = df_lt.groupby('status_label')['lead_time'].max().reset_index()['lead_time']
stats_lt['min_data'] = df_lt.groupby('status_label')['lead_time'].min().reset_index()['lead_time']

stats_lt['upper'] = stats_lt[['upper', 'max_data']].min(axis=1)
stats_lt['lower'] = stats_lt[['lower', 'min_data']].max(axis=1)

source_lt = ColumnDataSource(stats_lt)

# 2. Membuat Figure
p4 = figure(
    x_range=statuses, 
    height=400, 
    width=600, 
    title="Distribusi Lead Time (Hari) Berdasarkan Status Pembatalan",
    tools="pan, wheel_zoom, box_zoom, reset, save, hover",
    tooltips=[
        ("Status", "@status_label"),
        ("Median Lead Time", "@q2{0}"),
        ("IQR", "@iqr{0}")
    ]
)

# 3. Menambahkan Glyph (Box Plot Manual)
COLOR_PALETTE = ['#5cb85c', '#d9534f'] 

p4.vbar(x='status_label', top='q3', bottom='q2', width=0.7, source=source_lt, 
        line_color='black', fill_color=factor_cmap('status_label', palette=COLOR_PALETTE, factors=statuses))
p4.vbar(x='status_label', top='q2', bottom='q1', width=0.7, source=source_lt, 
        line_color='black', fill_color=factor_cmap('status_label', palette=COLOR_PALETTE, factors=statuses))

p4.segment(x0='status_label', y0='upper', x1='status_label', y1='q3', line_color="black", source=source_lt)
p4.segment(x0='status_label', y0='lower', x1='status_label', y1='q1', line_color="black", source=source_lt)

p4.rect(x='status_label', y='upper', width=0.2, height=0.01, line_color="black", fill_color="black", source=source_lt)
p4.rect(x='status_label', y='lower', width=0.2, height=0.01, line_color="black", fill_color="black", source=source_lt)


# 4. Styling
p4.xgrid.grid_line_color = None
p4.xaxis.axis_label = "Status Pembatalan"
p4.yaxis.axis_label = "Lead Time (Hari)"
p4.y_range.start = 0

show(p4)

### Deskripsi Grafik: Distribusi Lead Time Berdasarkan Status Pembatalan

Boxplot di atas menampilkan **perbandingan lead time (selang waktu antara pemesanan dan kedatangan)** antara dua kelompok pemesanan: **Tidak Dibatalkan** dan **Dibatalkan**. Visualisasi ini membantu memahami apakah pemesanan yang dilakukan jauh hari sebelumnya lebih berpotensi dibatalkan.

**Temuan Utama:**

- **Tidak Dibatalkan**
  - Median lead time relatif rendah.
  - Sebaran data cenderung lebih sempit.
  - Banyak pemesanan dilakukan dalam rentang waktu dekat (low lead time), yang umumnya lebih stabil dan jarang dibatalkan.

- **Dibatalkan**
  - Median lead time berada pada nilai yang lebih tinggi dibandingkan pemesanan yang tidak dibatalkan.
  - Variasi lead time lebih besar, dengan rentang interkuartil (IQR) yang lebih lebar.
  - Terdapat nilai-nilai ekstrem (outlier) dengan lead time sangat panjang, menunjukkan bahwa beberapa tamu membatalkan meskipun memesan jauh sebelum tanggal kedatangan.

**Interpretasi Umum:**
Pemesanan dengan **lead time yang lebih panjang cenderung lebih sering dibatalkan**. Hal ini wajar karena rencana tamu bisa berubah ketika interval antara pemesanan dan tanggal kedatangan terlalu jauh. Insight ini dapat membantu hotel merancang strategi mitigasi, seperti kebijakan deposit atau reminder booking untuk pemesanan ber-lead time panjang.


## Tabel Ringkasan – Lead Time & Status Pembatalan
Tabel ini menunjukkan perbedaan lead time antara booking dibatalkan dan tidak.

In [72]:
# =======================================================================
# TABEL RINGKASAN D.1: STATISTIK LEAD TIME BERDASARKAN STATUS PEMBATALAN
# =======================================================================

# Hitung metrik ringkasan Lead Time
lead_time_summary = df.groupby('is_canceled')['lead_time'].agg(
    Median_Lead_Time='median',
    Mean_Lead_Time='mean',
    Jumlah_Pemesanan='count'
).reset_index()

# Ubah nilai 'is_canceled' menjadi label yang deskriptif
lead_time_summary['Status Pembatalan'] = lead_time_summary['is_canceled'].map({
    0: 'Tidak Dibatalkan', 
    1: 'Dibatalkan'
})

# Format angka (pembulatan)
lead_time_summary['Median Lead Time'] = lead_time_summary['Median_Lead_Time'].round(0).astype(int)
lead_time_summary['Mean Lead Time'] = lead_time_summary['Mean_Lead_Time'].round(1)

# Pilih dan urutkan kolom akhir
final_lead_time_summary = lead_time_summary[[
    'Status Pembatalan', 
    'Median Lead Time', 
    'Mean Lead Time', 
    'Jumlah_Pemesanan'
]]

print("\n--- Tabel Ringkasan: Statistik Lead Time Berdasarkan Status Pembatalan ---")
print(final_lead_time_summary.to_markdown(index=False))


--- Tabel Ringkasan: Statistik Lead Time Berdasarkan Status Pembatalan ---
| Status Pembatalan   |   Median Lead Time |   Mean Lead Time |   Jumlah_Pemesanan |
|:--------------------|-------------------:|-----------------:|-------------------:|
| Tidak Dibatalkan    |                 45 |             80   |              75166 |
| Dibatalkan          |                113 |            144.8 |              44224 |


In [78]:
# =======================================================================
# VISUALISASI 5: DISTRIBUSI JUMLAH PEMESANAN BERDASARKAN SEGMEN PASAR
# =======================================================================

# 1. Persiapan Data
market_counts = df['market_segment'].value_counts().reset_index()
market_counts.columns = ['market_segment', 'count']

# Hitung persentase (untuk label)
total_market = market_counts['count'].sum()
market_counts['percentage'] = (market_counts['count'] / total_market) * 100
market_counts['percentage_label'] = market_counts['percentage'].apply(lambda p: f"{p:.1f}%")

# Urutkan berdasarkan jumlah (count) dari terbesar ke terkecil
market_counts = market_counts.sort_values(by='count', ascending=False)

# Atur Category Range untuk sumbu X
market_segments = market_counts['market_segment'].tolist()
source_market = ColumnDataSource(market_counts)

# 2. Membuat Figure
p5 = figure(
    x_range=market_segments, 
    height=400, 
    width=800, 
    title="Distribusi Jumlah Pemesanan Berdasarkan Segmen Pasar", 
    tools="pan, wheel_zoom, box_zoom, reset, save, hover",
    tooltips=[
        ("Segmen", "@market_segment"),
        ("Jumlah", "@count"),
        ("Persentase", "@percentage{0.1f}%")
    ]
)

# 3. Menambahkan Glyph (Bar Plot)
COLOR = Category10[3][2] 

p5.vbar(x='market_segment', top='count', width=0.8, source=source_market, 
        line_color='white', fill_color=COLOR)

# Menambahkan Label Persentase di atas Batang
p5.text(x='market_segment', y='count', 
        text='percentage_label', 
        text_align="center", 
        text_baseline="bottom",
        source=source_market,
        text_font_size="10pt"
        ) 

# 4. Styling
p5.xaxis.major_label_orientation = 0.8
p5.xgrid.grid_line_color = None
p5.y_range.start = 0
p5.xaxis.axis_label = "Segmen Pasar"
p5.yaxis.axis_label = "Total Jumlah Pemesanan"

show(p5)

### Deskripsi Grafik: Distribusi Jumlah Pemesanan Berdasarkan Segmen Pasar

Grafik batang di atas menampilkan **proporsi dan jumlah total pemesanan dari berbagai segmen pasar**. Visualisasi ini membantu memahami dari mana sebagian besar pelanggan berasal serta segmen mana yang menjadi kontributor utama terhadap volume pemesanan hotel.

**Temuan Utama:**

- **Online TA (47.3%)**  
  Merupakan segmen terbesar dengan hampir setengah total pemesanan. Hal ini menunjukkan ketergantungan yang cukup besar pada platform pemesanan online.

- **Offline TA/TO (20.3%)**  
  Segmen kedua terbesar, menunjukkan bahwa agen perjalanan tradisional masih memegang peran penting dalam penjualan kamar.

- **Groups (16.8%)**  
  Kontributor signifikan, biasanya berasal dari pemesanan rombongan seperti acara perusahaan, tur, atau kegiatan organisasi.

- **Direct Booking (10.6%)**  
  Persentase yang cukup baik, namun masih jauh tertinggal dari OTA. Ini dapat menjadi peluang untuk meningkatkan direct sales melalui promosi atau loyalty program.

- **Corporate (4.4%)**  
  Segmen bisnis dengan kontribusi kecil namun stabil.

- **Complementary, Aviation, dan Undefined (<1%)**  
  Segmen minor dengan volume pemesanan yang sangat kecil.

**Interpretasi Umum:**
Sebagian besar pemesanan berasal dari kanal online, menandakan bahwa **OTA adalah sumber pelanggan utama**. Sementara itu, pemesanan langsung dan segmen korporat masih dapat ditingkatkan melalui strategi pemasaran khusus, program loyalti, atau kerja sama perusahaan. Grafik ini membantu hotel fokus pada segmen yang paling berpengaruh sekaligus menemukan potensi pertumbuhan pada segmen lain.

----


## Kesimpulan Antar-Visualisasi

Secara keseluruhan, pola pemesanan hotel dipengaruhi oleh musim, kanal pemesanan, dan lead time. Tingkat pembatalan yang tinggi (37%) berkaitan dengan lead time panjang. ADR menunjukkan bahwa Resort Hotel memiliki variasi harga lebih besar. Tren pemesanan bulanan mencapai puncak pada pertengahan tahun, dan OTA menjadi sumber pemesanan terbesar. Seluruh temuan ini memberikan gambaran menyeluruh tentang dinamika permintaan hotel.

---


## Insight Bisnis & Rekomendasi

## 1. Mengurangi Tingkat Pembatalan 
* Gunakan **kebijakan deposit** untuk pemesanan dengan lead time panjang.
* Kirim **reminder otomatis** sebelum tamu tiba.
* Terapkan **free cancellation hanya sampai batas tertentu**.
* Beri insentif (*confirm & save*) untuk pelanggan yang konfirmasi ulang.
  
## 2. Optimalisasi Pricing 
* **City Hotel:** gunakan harga stabil + paket bundling.
* **Resort Hotel:** cocok untuk **dynamic pricing** berbasis musim dan permintaan.
* Manfaatkan ADR outlier sebagai peluang untuk segmen premium.

## 3. Meningkatkan Direct Booking 
* Tawarkan **best price guarantee** di situs hotel.
* Gunakan **loyalty program** untuk repeat guest.
* Tingkatkan UI/UX website agar proses booking cepat dan mudah
* Lakukan remarketing pada pengunjung website yang belum booking.
  
## 4. Manajemen Kapasitas Berdasarkan Musim 
* Siapkan staf tambahan pada periode peak season (Mei–Agustus).
* Mulai kampanye promosi 2–3 bulan sebelum peak.
* Gunakan strategi promosi off-season seperti:
    * long-stay package
    * corporate package
    * family weekend promo
      
## 5. Segmentasi & Personalisasi 
* Fokuskan iklan pada OTA dan TA/TO sebagai segmen terbesar.
* Buat promo khusus untuk pemesanan grup.
* Perkuat kerja sama perusahaan untuk meningkatkan segmen corporate.